<a href="https://colab.research.google.com/github/RegiRau/Vein_Segmentation/blob/main/Hyperparameters_Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install wandb --upgrade

In [ ]:
import wandb

wandb.login()

True

In [ ]:
sweep_config = {
    'method': 'random'
    }

In [ ]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
import math

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
      'batch_size':{
          'distribution': 'uniform',
          'min': 3,
          'max': 32,

      }
    # 'batch_size': {
    #     # integers between 32 and 256
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform',
    #     'q': 1,
    #     'min': math.log(32),
    #     'max': math.log(256),
    #   }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 1}
    })

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'uniform', 'max': 32, 'min': 3},
                'dropout': {'values': [0.1, 0.2, 0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Vein_Segmentation_Hyperparameters", entity="regirau")

Create sweep with ID: v7tlljv7
Sweep URL: https://wandb.ai/regirau/Vein_Segmentation_Hyperparameters/sweeps/v7tlljv7


# Build Dataset

In [ ]:
!unzip /content/gdrive/MyDrive/Vein_Dataset_Resized.zip -d /content/

Archive:  /content/gdrive/MyDrive/Vein_Dataset_Resized.zip
   creating: /content/Vein_Dataset_Resized/images/
  inflating: /content/Vein_Dataset_Resized/images/001_image01_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image01_R.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image02_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image02_R.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image03_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image03_R.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image04_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image04_R.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image05_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image05_R.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image06_L.png  
  inflating: /content/Vein_Dataset_Resized/images/001_image06_R.png  
  inflating: /content/Vein_Dataset_Resized/images/

In [ ]:
import tensorflow as tf
dataset_images = tf.data
dataset_labels = tf.data
SIZE_X = 1632
SIZE_Y = 1216


dataset_images = tf.keras.utils.image_dataset_from_directory("Vein_Dataset_Resized/images", labels=None, label_mode=None, color_mode="grayscale", batch_size=1, image_size=(SIZE_Y, SIZE_X), shuffle=False, )
dataset_labels = tf.keras.utils.image_dataset_from_directory("Vein_Dataset_Resized/labels", labels=None, label_mode=None, color_mode="grayscale", batch_size=1, image_size=(SIZE_Y, SIZE_X), shuffle=False, )
normalization_layer = tf.keras.layers.Rescaling(1./255)
dataset_labels = dataset_labels.map(lambda x: (normalization_layer(x)))

Vein_Dataset = tf.data.Dataset.zip((dataset_images, dataset_labels))

Found 900 files belonging to 1 classes.
Found 900 files belonging to 1 classes.


In [ ]:
Vein_Dataset.shuffle(buffer_size = 1000)

Dataset_Size = 900

train_size = int(0.7 * Dataset_Size)
val_size = int(0.15 * Dataset_Size)
test_size = int(0.15 * Dataset_Size)

train_dataset = Vein_Dataset.take(train_size)
test_dataset = Vein_Dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [ ]:
import tensorflow as tf

def loader():
  Vein_Dataset.shuffle(buffer_size = 1000)
  Dataset_Size = 900

  train_size = int(0.7 * Dataset_Size)
  val_size = int(0.15 * Dataset_Size)
  test_size = int(0.15 * Dataset_Size)

  train_dataset = Vein_Dataset.take(train_size)
  test_dataset = Vein_Dataset.skip(train_size)
  val_dataset = test_dataset.skip(test_size)
  test_dataset = test_dataset.take(test_size)

  return train_dataset, val_dataset


In [ ]:
# import numpy as np

# run_number = 'run_2'
# x_val = np.load('/content/gdrive/MyDrive/train_test_split/' + run_number + '/training1/x_val.npy')
# y_val = np.load('/content/gdrive/MyDrive/train_test_split/' + run_number + '/training1/y_val.npy')
# x_train = np.load('/content/gdrive/MyDrive/train_test_split/' + run_number + '/training1/x_train.npy')
# y_train = np.load('/content/gdrive/MyDrive/train_test_split/' + run_number + '/training1/y_train.npy')

# print('Shape of x_train: ', x_train.shape)
# print('Shape of y_train: ', y_train.shape)
# print('Shape of x_val: ', x_val.shape)
# print('Shape of y_val: ', y_val.shape)

Shape of x_train:  (135, 1216, 1632, 1)
Shape of y_train:  (135, 1216, 1632, 1)
Shape of x_val:  (135, 1216, 1632, 1)
Shape of y_val:  (135, 1216, 1632, 1)


In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
print("TensorFlow **IS** using the GPU")

TensorFlow **IS** using the GPU


# Define the Model

In [ ]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K


#convolutional block
def conv_block(x, kernelsize, filters, dropout, batchnorm=False): 
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(x)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(conv)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    return conv

        
    #skip connection    
    shortcut = layers.Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)
    shortcut = layers.Activation("relu")(shortcut)
    respath = layers.add([shortcut, conv2])       
    return respath


#gating signal for attention unit
def gatingsignal(input, out_size, batchnorm=False):
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

#attention unit/block based on soft attention
def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), kernel_initializer='he_normal', padding='same')(x) 
    shape_theta_x = K.int_shape(theta_x)
    phi_g = layers.Conv2D(inter_shape, (1, 1), kernel_initializer='he_normal', padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3), strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]), kernel_initializer='he_normal', padding='same')(phi_g)
    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), kernel_initializer='he_normal', padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg) 
    upsample_psi = layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': shape_x[3]})(upsample_psi)                          
    y = layers.multiply([upsample_psi, x])
    result = layers.Conv2D(shape_x[3], (1, 1), kernel_initializer='he_normal', padding='same')(y)
    attenblock = layers.BatchNormalization()(result)
    return attenblock

#Attention U-NET
def attentionunet(input_shape, dropout=0.2, batchnorm=True):
    
    filters = [16, 32, 64, 128, 256]
    kernelsize = 3
    upsample_size = 2

    inputs = layers.Input(input_shape) 

    # Downsampling layers    
    dn_1 = conv_block(inputs, kernelsize, filters[0], dropout, batchnorm)
    pool_1 = layers.MaxPooling2D(pool_size=(2,2))(dn_1)
    
    dn_2 = conv_block(pool_1, kernelsize, filters[1], dropout, batchnorm)
    pool_2 = layers.MaxPooling2D(pool_size=(2,2))(dn_2)
    
    dn_3 = conv_block(pool_2, kernelsize, filters[2], dropout, batchnorm)
    pool_3 = layers.MaxPooling2D(pool_size=(2,2))(dn_3)
    
    dn_4 = conv_block(pool_3, kernelsize, filters[3], dropout, batchnorm)
    pool_4 = layers.MaxPooling2D(pool_size=(2,2))(dn_4)
    
    dn_5 = conv_block(pool_4, kernelsize, filters[4], dropout, batchnorm)

    # Upsampling layers    
    gating_5 = gatingsignal(dn_5, filters[3], batchnorm)
    att_5 = attention_block(dn_4, gating_5, filters[3])
    up_5 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(dn_5)
    up_5 = layers.concatenate([up_5, att_5], axis=3)
    up_conv_5 = conv_block(up_5, kernelsize, filters[3], dropout, batchnorm)
    
    gating_4 = gatingsignal(up_conv_5, filters[2], batchnorm)
    att_4 = attention_block(dn_3, gating_4, filters[2])
    up_4 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_5)
    up_4 = layers.concatenate([up_4, att_4], axis=3)
    up_conv_4 = conv_block(up_4, kernelsize, filters[2], dropout, batchnorm)
   
    gating_3 = gatingsignal(up_conv_4, filters[1], batchnorm)
    att_3 = attention_block(dn_2, gating_3, filters[1])
    up_3 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_4)
    up_3 = layers.concatenate([up_3, att_3], axis=3)
    up_conv_3 = conv_block(up_3, kernelsize, filters[1], dropout, batchnorm)
    
    gating_2 = gatingsignal(up_conv_3, filters[0], batchnorm)
    att_2 = attention_block(dn_1, gating_2, filters[0])
    up_2 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_3)
    up_2 = layers.concatenate([up_2, att_2], axis=3)
    up_conv_2 = conv_block(up_2, kernelsize, filters[0], dropout, batchnorm)
    
    conv_final = layers.Conv2D(1, kernel_size=(1,1))(up_conv_2)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    outputs = layers.Activation('sigmoid')(conv_final)  

    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.summary()       
    return model 

# Define Evaluation Metrices

In [ ]:
from tensorflow.keras import backend as K
from sklearn.metrics import jaccard_score,confusion_matrix


def IoU_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def IoU_loss(y_true, y_pred):
    return -IoU_coef(y_true, y_pred)
 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
  
def accuracy(y_true, y_pred):
    cm = confusion_matrix(y_true.flatten(),y_pred.flatten(), labels=[0, 1])
    acc = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
    return acc
  
def IoU(y_true, y_pred, labels = [0, 1]):
   IoU = []
   for label in labels:
      jaccard = jaccard_score(y_pred.flatten(),y_true.flatten(), pos_label=label, average='binary')
      IoU.append(jaccard)     
   return np.mean(IoU) 


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

def build_optimizer(optim, learning_rate):
    if optim == "sgd":
        optimizer = SGD(lr=learning_rate, momentum=0.9)
    elif optim == "adam":
        optimizer = Adam(learning_rate=learning_rate)

    return optimizer

In [ ]:
from tensorflow.keras.optimizers import Adam
import numpy as np

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        IMG_CHANNELS = 1
        SIZE_X = 1632
        SIZE_Y = 1216
        IMG_HEIGHT = SIZE_Y
        IMG_WIDTH = SIZE_X

        input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

        model = attentionunet(input_shape, config.dropout)
        model.compile(optimizer = build_optimizer(config.optimizer, config.learning_rate), loss= IoU_loss, metrics= ['accuracy', IoU_coef])

        train_dataset, val_dataset = loader()

        history = model.fit(train_dataset, 
                            epochs = config.epochs,
                            batch_size = config.batch_size,
                            verbose=1,
                            validation_data=val_dataset)
            
        loss = np.array(history.history['loss'])
        avg_loss = np.mean(loss)
        wandb.log({"loss" : avg_loss})

In [ ]:
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: jzd3jz29 with config:
wandb: 	batch_size: 9.000103048680307
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	learning_rate: 0.023810326825610365
wandb: 	optimizer: sgd


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1216, 1632,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 1216, 1632,   160         ['input_1[0][0]']                
                                16)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 1216, 1632,   64         ['conv2d[0][0]']                 
 alization)                     16)                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


630/630 [==============================] - 414s 651ms/step - loss: -0.0599 - accuracy: 0.6273 - IoU_coef: 0.0599 - val_loss: -0.0931 - val_accuracy: 0.6194 - val_IoU_coef: 0.0931



loss,▁
loss,-0.05994


wandb: Agent Starting Run: b61biliq with config:
wandb: 	batch_size: 21.837383262169
wandb: 	dropout: 0.1
wandb: 	epochs: 1
wandb: 	learning_rate: 0.05571522160618629
wandb: 	optimizer: sgd


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1216, 1632,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 1216, 1632,   160         ['input_1[0][0]']                
                                16)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 1216, 1632,   64         ['conv2d[0][0]']                 
 alization)                     16)                                                           

loss,▁
loss,-0.26483
